In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

############################################
# User-Based Collaborative Filtering
#############################################

## Adım 1: Veri Setinin Hazırlanması
## Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
## Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
## Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
## Adım 5: Weighted Average Recommendation Score'un Hesaplanması
## Adım 6: Çalışmanın Fonksiyonlaştırılması



#############################################
## Adım 1: Veri Setinin Hazırlanması
#############################################

In [ ]:
# Birinci yol:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
    rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()

In [ ]:
# İkinci yol:
# import pickle
# user_movie_df = pickle.load(open('user_movie_df.pkl', 'rb'))

import pandas as pd
pd.set_option('display.max_columns', 5)


#############################################
## Adım 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
#############################################

In [ ]:
#rastgele bir kullanıcı seçiyorum
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
random_user

In [ ]:
# sinanın izlediği filmlere giteceğim.
# sinanın izlediği çok film yoksa, item-based  yada content-based yapacağız. Trendlerden tavsiye edeceğim.

# canlı sistemlerde dikkat, 

#random olarak seçtiğim kişiye
random_user_df = user_movie_df[user_movie_df.index == random_user]

# bana sinanın izlediği filmler lazım. Ortak karşılaştırma havuzu lazım. 
#tolist
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
#sinanın izlediği filmler
movies_watched

In [ ]:
# var olanları filmelere kaç puan vermiş
user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns == "Schindler's List (1993)"]


In [ ]:
# kaç film izlemiş
len(movies_watched)

#############################################
## Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
#############################################

In [ ]:
# fancy indeks ile değişkenleri seçme
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

In [ ]:
# satırlar, sinan ile sinan ile aynı aynı filmleri izlmişler
# en az bir tanesi aynı filmi izlmişler
# sinan ile aynı filmmi izlimiş kişiler, aynı sonuca götürmez. Bir eşik gerekiyor.
movies_watched_df.shape

In [ ]:
movies_watched_df.T.head(10)

In [ ]:
# her bir kullanıcı için boş değilse tru false çevirme
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head(8)

In [ ]:
#user id leri değişkene değiştirdik, indexten kurtardık
user_movie_count = user_movie_count.reset_index()

In [ ]:
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head(14)

In [ ]:
#bana belrili bir sayının üzerinden izleyen kişiler lazım
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)

In [ ]:
#tamamen aynı filmi izleyenler kaç kişi var.
user_movie_count[user_movie_count["movie_count"] == 33].count()

In [ ]:
# en 20 filmi birlikte beğenleri getir benim kullanıcımla
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]
users_same_movies.head()

In [ ]:
users_same_movies.count()

In [ ]:
# perc = len(movies_watched) * 60 / 100
# users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]



#############################################
## Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi
#############################################

## Bunun için 3 adım gerçekleştireceğiz:
 1. Sinan ve diğer kullanıcıların verilerini bir araya getireceğiz.
 2. Korelasyon df'ini oluşturacağız.
 3. En benzer bullanıcıları (Top Users) bulacağız

In [ ]:
#sinan ve diğerlerini yan yana getirdim. isin() belirli bir listeyegöre seçme yapar

final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)],
                      random_user_df[movies_watched]])

final_df.head()

In [ ]:
#sinanı da ekledik. 
## 33tane de film var. kullancıların, izledikleri filmlerin 20den fazlası sinan ile aynı film izlmişler.
final_df.shape

In [ ]:
final_df.T.corr()

In [ ]:
# unstackten ötürü dblicate geliyor. onları uçurmak gerekiyor.
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df

In [ ]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.head()

In [ ]:
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df.head()

In [ ]:
corr_df = corr_df.reset_index()
corr_df.head()

In [ ]:
# Sinan ile yüzde 65 ve üzeri korelasyona sahip kullanıcılar:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True) #iki değişkeni seçtik.
top_users.head()
#sinan ile korelasyonu olan kişiler

In [ ]:
#sinanla en yüksek korelasyona sahip kişiler kimler?
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.head()

In [ ]:
# değişkenleri tekrardan isimlendiriyorum
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.head()

In [ ]:
# bu kullanıcılar, hangi filme kaç puan vermiş?
rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
# skorlama problemini görebildiniz mi? ölçüm problemleri
# sinanın bir kullancı ile benzer puan verebilme alışkanlığı olabilir. 
# biri yüksek veriyor, diğeri düşük puan veriyor.
# korelasyonun ve rating ağırlığını nasıl ayarlayacağız?

#sinanla en yüksek korelasyona sahip kişiler
top_users_ratings.head()

#############################################
## Adım 5: Weighted Average Recommendation Score'un Hesaplanması
#############################################

1.  skorlama problemini görebildiniz mi? ölçüm problemleri
1.  sinanın bir kullancı ile benzer puan verebilme alışkanlığı olabilir. 
1.  biri yüksek veriyor, diğeri düşük puan veriyor.
1.  korelasyonun ve rating ağırlığını nasıl ayarlayacağız?

Üniversite sınavında derslerin katsayıları  olur. Türkçe ve mat katsayıları en fazla puan getirir.

**# korelasyonu düşük olanlar daha çoık kırpacaktır.**

- Her yerde bulanamayanlar serisi

In [ ]:


# weighted_rating'in hesaplanması.
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

In [ ]:
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

In [ ]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = recommendation_df.reset_index()

recommendation_df[["movieId"]].nunique()

In [ ]:
# dörten büyük ağırlıkları getir
recommendation_df[recommendation_df["weighted_rating"] > 4]

In [ ]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 4]

# idler ile film isimlerini getirir misin?
movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])

In [ ]:
movies_to_be_recommend

#############################################
## Adım 6: Çalışmanın Fonksiyonlaştırılması
#############################################

In [ ]:


def user_based_recommender():
    import pickle
    import pandas as pd
    user_movie_df = pickle.load(open('user_movie_df.pkl', 'rb'))
    random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
    random_user_df = user_movie_df[user_movie_df.index == random_user]
    movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
    movies_watched_df = user_movie_df[movies_watched]
    user_movie_count = movies_watched_df.T.notnull().sum()
    user_movie_count = user_movie_count.reset_index()
    user_movie_count.columns = ["userId", "movie_count"]
    users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]

    final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)],
                          random_user_df[movies_watched]])

    corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
    corr_df = pd.DataFrame(corr_df, columns=["corr"])
    corr_df.index.names = ['user_id_1', 'user_id_2']
    corr_df = corr_df.reset_index()

    top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
        ["user_id_2", "corr"]].reset_index(drop=True)

    top_users = top_users.sort_values(by='corr', ascending=False)
    top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
    rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
    top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
    top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

    recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
    recommendation_df = recommendation_df.reset_index()

    movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 4]
    movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
    return movies_to_be_recommend.merge(movie[["movieId", "title"]])


user_based_recommender()